### 벡터 스토어
: LangChain에서 Vector Store(벡터 저장소) 는 텍스트, 문서 등의 임베딩 벡터를 저장하고 검색하는 역할을 한다.

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

### [1] FAISS
FAISS(Facebook AI Similarity Search)는 Facebook AI Research에서 개발한 오픈소스 벡터 검색 라이브러리로, <br>
대규모 벡터(임베딩) 데이터에서 유사한 항목을 빠르게 검색하기 위해 사용된다.<br>
LangChain이나 RAG(Retrieval-Augmented Generation) 시스템에서 문서 검색용 벡터 저장소(Vector Store)로 자주 사용된다.

In [4]:
# from_texts 사용
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 1. 임베딩 모델
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 텍스트 리스트
texts = [
    "서울의 날씨는 맑습니다.",
    "부산은 해운대가 유명합니다.",
    "제주는 바람이 셉니다."
]

# 3. from_texts()로 벡터 저장소 생성
faiss_store = FAISS.from_texts(texts,embedding)

# 4. 검색
query = "바람이 부는 지역"
results = faiss_store.similarity_search(query,k=2)

for r in results:
    print(r.page_content)

# 문자열을 바로 임베딩하여 인덱스 생성
# 메타데이터는 자동으로 생성되지 않음 ({'source': None} 기본값)

제주는 바람이 셉니다.
서울의 날씨는 맑습니다.


In [5]:
results

[Document(id='e07d6ba9-fa35-47f6-bb7d-e4ffef1f81cd', metadata={}, page_content='제주는 바람이 셉니다.'),
 Document(id='7fdab6ae-6204-4f11-91a6-f4ec954de7c8', metadata={}, page_content='서울의 날씨는 맑습니다.')]

In [6]:
# from_documents 사용
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

# 1. 임베딩 모델
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 문서 + 메타데이터
docs = [
    Document(page_content="서울의 날씨는 맑습니다.", metadata={"city": "Seoul"}),
    Document(page_content="부산은 해운대가 유명합니다.", metadata={"city": "Busan"}),
    Document(page_content="제주는 바람이 셉니다.", metadata={"city": "Jeju"})
]

# 3. from_documents()로 벡터 저장소 생성
faiss_store = FAISS.from_documents(docs,embedding)

# 4. 검색 (쿼리 + 결과 메타데이터 확인)
query = "해운대 해변이 있는 곳"
results = faiss_store.similarity_search(query,k=2)

for r in results:
    print(f"내용: {r.page_content}, 도시: {r.metadata['city']}")

# Document 객체에는 본문(page_content) 과 메타데이터(metadata)가 포함됨
# PDF나 웹페이지 등 “출처가 있는 데이터”를 다룰 때 유용함
# 검색 결과에서도 metadata를 함께 확인할 수 있음

내용: 부산은 해운대가 유명합니다., 도시: Busan
내용: 제주는 바람이 셉니다., 도시: Jeju


In [8]:
# 인덱스 저장과 읽어오기
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 1) 임베딩 모델 정의
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 2) 벡터 저장소 생성
texts = [
    "서울의 봄은 벚꽃이 피어서 아름답다.",
    "부산의 해운대는 여름에 많은 사람이 찾는다.",
    "제주도는 겨울에도 따뜻한 편이다."
]

faiss_store = FAISS.from_texts(texts, embedding)

# 3) 인덱스 저장
faiss_store.save_local("faiss_store")
print(" 인덱스 저장 완료 (폴더명: faiss_store)")

 인덱스 저장 완료 (폴더명: faiss_store)


In [9]:
# 4) 인덱스 로드 (pickle 역직렬화 허용)
loaded_store = FAISS.load_local(
    "faiss_store",
    embedding,
    allow_dangerous_deserialization=True  # 직접 만든 인덱스일 경우만 True 설정
)
print(" 인덱스 로드 완료")

# 5) 검색 수행
query = "겨울에도 따뜻한 지역"
results = loaded_store.similarity_search(query, k=2)

# 6) 결과 출력
for i, r in enumerate(results, 1):
    print(f"{i}. {r.page_content}")

 인덱스 로드 완료
1. 제주도는 겨울에도 따뜻한 편이다.
2. 서울의 봄은 벚꽃이 피어서 아름답다.


In [12]:
# 두 개의 인덱스를 병합하기
# faiss_merge_example.py
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델 정의
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 첫 번째 인덱스 (서울 관련)
texts1 = [
    "서울의 봄은 벚꽃이 아름답다.",
    "한강공원은 서울 시민의 휴식 공간이다."
]
store1 = FAISS.from_texts(texts1, embedding)

# 두 번째 인덱스 (부산 관련)
texts2 = [
    "부산의 해운대는 여름에 관광객이 많다.",
    "광안대교는 부산의 대표적인 명소이다."
]
store2 = FAISS.from_texts(texts2, embedding)

# 인덱스 병합
store1.merge_from(store2)
print("두 개의 인덱스 병합 완료")

# 병합된 인덱스 검색
query = "여름 관광지"
results = store1.similarity_search(query, k=3)

for i, r in enumerate(results, 1):
    print(f"{i}. {r.page_content}")

두 개의 인덱스 병합 완료
1. 부산의 해운대는 여름에 관광객이 많다.
2. 서울의 봄은 벚꽃이 아름답다.
3. 한강공원은 서울 시민의 휴식 공간이다.


In [13]:
# 병합된 인덱스를 로컬에 저장
store1.save_local("merged_faiss_store")

# 다시 불러오기
loaded = FAISS.load_local(
    "merged_faiss_store",
    embedding,
    allow_dangerous_deserialization=True
)
print(" 병합된 인덱스 로드 완료")

 병합된 인덱스 로드 완료


### [2] Chroma
: Chroma는 2023년 2월경 미국의 스타트업 Chroma가 발표한 오픈소스 벡터 데이터베이스로,
개발자 친화적인 로컬 RAG 기반 시스템 구축용 DB이다.

In [14]:
from langchain_chroma import Chroma

# 1. 임베딩 모델
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 문장 리스트
texts = [
    "서울의 봄은 벚꽃이 아름답다.",
    "부산의 해운대는 여름에 관광객이 많다.",
    "제주도는 겨울에도 따뜻하다."
]

# 3. Chroma 벡터 저장소 생성 (자동으로 로컬 DB 생성)
chroma_store = Chroma.from_texts(
    texts=texts,
    embedding=embedding,
    collection_name="korea_travel"
)

# 4. 검색
query = "겨울에 따뜻한 지역"
results = chroma_store.similarity_search(query,k=2)

for r in results:
    print(r.page_content)

제주도는 겨울에도 따뜻하다.
서울의 봄은 벚꽃이 아름답다.


In [15]:
# 저장/로드 명시적 설정
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

texts = [
    "서울의 봄은 벚꽃이 아름답다.",
    "부산의 해운대는 여름에 관광객이 많다.",
    "제주도는 겨울에도 따뜻하다."
]

# 저장 경로 지정 (자동 영구 저장)
chroma_store = Chroma.from_texts(
    texts=texts,
    embedding=embedding,
    collection_name="travel_korea",
    persist_directory="./chroma_db"
)

print("Chroma 인덱스 자동 저장 완료")

# 불러오기 (persist_directory 지정 필수)
loaded_store = Chroma(
    collection_name="travel_korea",
    embedding_function=embedding,
    persist_directory="./chroma_db"
)
print("Chroma 인덱스 로드 완료")

# 검색
query = "겨울에 따뜻한 지역"
results = loaded_store.similarity_search(query, k=2)
for r in results:
    print(r.page_content)

Chroma 인덱스 자동 저장 완료
Chroma 인덱스 로드 완료
제주도는 겨울에도 따뜻하다.
서울의 봄은 벚꽃이 아름답다.


In [16]:
# 메타데이터 활용 (필터링 검색)
from langchain_core.documents import Document

docs = [
    Document(page_content="서울의 벚꽃 명소는 여의도입니다.", metadata={"city": "Seoul"}),
    Document(page_content="부산의 해운대는 여름에 인기가 많습니다.", metadata={"city": "Busan"}),
    Document(page_content="제주는 겨울에도 따뜻합니다.", metadata={"city": "Jeju"}),
]

chroma = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    collection_name="travel_guide",
    persist_directory="./chroma_db"
)

# 특정 도시만 필터링
results = chroma.similarity_search("따뜻한 지역", k=2, filter={"city": "Jeju"})
for r in results:
    print(r.page_content, r.metadata)

제주는 겨울에도 따뜻합니다. {'city': 'Jeju'}


In [17]:
# 문서 추가 및 문서 삭제
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

embedding = OpenAIEmbeddings(model="text-embedding-3-small")
persist_dir = "./chroma_db"

# 초기 문서 3개
docs = [
    Document(page_content="서울의 봄은 벚꽃이 아름답다.", metadata={"city": "Seoul"}),
    Document(page_content="부산의 해운대는 여름에 인기가 많다.", metadata={"city": "Busan"}),
    Document(page_content="제주는 겨울에도 따뜻한 편이다.", metadata={"city": "Jeju"}),
]

chroma_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    collection_name="korea_travel",
    persist_directory=persist_dir,
)
print(" 초기 문서 3개 추가 및 저장 완료")

# 기존 DB 로드
loaded_store = Chroma(
    collection_name="korea_travel",
    embedding_function=embedding,
    persist_directory=persist_dir,
)

# 새 문서 추가
new_docs = [
    Document(page_content="강릉은 바다와 커피로 유명하다.", metadata={"city": "Gangneung"}),
    Document(page_content="속초는 설악산이 가까워 등산객이 많다.", metadata={"city": "Sokcho"}),
]
loaded_store.add_documents(new_docs)
print(" 새 문서 2개 추가 완료")

# 검색
query = "바다 여행지"
results = loaded_store.similarity_search(query, k=5)

print("\n [검색 결과]")
for r in results:
    city = r.metadata.get("city", "정보 없음")   # ← 안전한 접근
    print(f"- {r.page_content} ({city})")

# 특정 조건 삭제
loaded_store.delete(where={"city": "Seoul"})
print("\n 'Seoul' 문서 삭제 완료")

# 삭제 후 확인
results = loaded_store.similarity_search("여행", k=5)
print("\n [삭제 후 남은 문서]")
for r in results:
    city = r.metadata.get("city", "정보 없음")
    print(f"- {r.page_content} ({city})")

 초기 문서 3개 추가 및 저장 완료
 새 문서 2개 추가 완료

 [검색 결과]
- 제주는 겨울에도 따뜻한 편이다. (Jeju)
- 부산의 해운대는 여름에 인기가 많다. (Busan)
- 서울의 봄은 벚꽃이 아름답다. (Seoul)
- 강릉은 바다와 커피로 유명하다. (Gangneung)
- 속초는 설악산이 가까워 등산객이 많다. (Sokcho)

 'Seoul' 문서 삭제 완료

 [삭제 후 남은 문서]
- 부산의 해운대는 여름에 인기가 많다. (Busan)
- 제주는 겨울에도 따뜻한 편이다. (Jeju)
- 강릉은 바다와 커피로 유명하다. (Gangneung)
- 속초는 설악산이 가까워 등산객이 많다. (Sokcho)


In [18]:
#  전체 문서 삭제
loaded_store.reset_collection()  # 벡터 저장소의 컬렉션을 초기화
print(" 모든 문서 삭제 완료")

#  삭제 후 확인
results = loaded_store.similarity_search("여행", k=5)
print(f"\n 남은 문서 개수: {len(results)}")

 모든 문서 삭제 완료

 남은 문서 개수: 0


### [3] PostgreSQL + pgvector